In [ ]:
import sys
sys.path.insert(0, '/mnt/nas/users/mjm/LDPUts')

from discretizer import discretizer
from client import client
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import matplotlib.pyplot as plt
from server import server_ell2, server_multinomial_genRR, server_multinomial_bitflip
from data_generator import data_generator
from discretizer import discretizer
import time
import numpy as np
from scipy.stats import chi2
from utils import chi_sq_dist

In [ ]:
n_test = 200
n_permutation = 199
significance_level = 0.05
alphabet_size = 40
privacy_level = 1
###################
sample_size = 8000
###################

data_gen = data_generator(device)

bump_size = 0.005
p = torch.tensor([1/alphabet_size]).repeat(alphabet_size)


p = p.add(
    torch.remainder(
    torch.tensor(range(alphabet_size)),
    2
    ).add(-1/2).mul(2).mul(bump_size)
)



LDPclient = client(device, privacy_level)
server_elltwo = server_ell2(device, privacy_level)
server_genrr = server_multinomial_genRR(device, privacy_level)
server_bitflip = server_multinomial_bitflip(device, privacy_level)

p_value_array = np.zeros([n_test, 3])

In [ ]:
t = time.time()
for i in range(n_test):
    print(f"{i+1}th test")
    torch.manual_seed(i)
    data_y = data_gen.generate_multinomial_data(p, sample_size)
    data_z = data_gen.generate_multinomial_data(p, sample_size)
    
    LDPclient.load_data_multinomial(data_y, data_z, alphabet_size)
    data_list_lapu_y, data_list_lapu_z = LDPclient.release_LapU()
    data_list_genrr_y, data_list_genrr_z = LDPclient.release_genRR()
    data_list_bitflip_y, data_list_bitflip_z = LDPclient.release_bitFlip()
    
    server_elltwo.load_private_data_multinomial(data_list_lapu_y, data_list_lapu_z, alphabet_size)
    p_value_array[i,0] = server_elltwo.release_p_value_permutation(n_permutation)
 
    server_genrr.load_private_data_multinomial(data_list_genrr_y, data_list_genrr_z, alphabet_size)
    p_value_array[i,1] = server_genrr.release_p_value()
     
    server_bitflip.load_private_data_multinomial(data_list_bitflip_y, data_list_bitflip_z, alphabet_size)
    p_value_array[i,2] = server_bitflip.release_p_value()
elapsed = time.time() - t
print(elapsed)

In [ ]:
print(
   # f"small chi-square distance\n"+
        f"p = {p}\n"+
        f"privacy level = {privacy_level}"
)

print(
    f"sample size = {sample_size},\n"+
    f"type I error estimate =\n"+
    f"    {sum( p_value_array[:,0]<0.05)/n_test }(LapU, ell2)\n"+
    f"    {sum( p_value_array[:,1]<0.05)/n_test }(GenRR, chi-sq)\n"+
    f"    {sum( p_value_array[:,2]<0.05)/n_test }(bitflip, ch-sq-proj)\n"
)
with open('/mnt/nas/users/mjm/LDPUts/simul/type_1/hi_dim/n_6000_alpha_1_type_1.npy', 'wb') as f:
    np.save(f, p_value_array)